# TSP
### 都市の位置を乱数で生成

In [ ]:
import numpy as np

def gen_random_tsp(ncity: int):
    # 座標
    locations = np.random.uniform(size=(ncity, 2))

    # 距離行列
    all_diffs = np.expand_dims(locations, axis=1) - np.expand_dims(locations, axis=0)
    distances = np.sqrt(np.sum(all_diffs ** 2, axis=-1))

    return locations, distances

### 都市のプロット

In [ ]:
import matplotlib.pyplot as plt

def show_plot(locs: np.ndarray):
    plt.figure(figsize=(7, 7))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.scatter(*zip(*locations))
    plt.show()


# 都市数
ncity = 70
locations, distances = gen_random_tsp(ncity)

In [ ]:
show_plot(locations)

### 決定変数生成
２値変数 $q_{n,i}$ : $n$ 順番のインデックス、$i$ 都市のインデックス

In [ ]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
# ２値変数 q_{i,j} : 都市 i から都市 j への経路を選択するとき 1
q = gen.array(ncity, ncity)

### コスト関数
都市 $i$ と都市 $j$ の距離を $d_{ij}$、都市数を $N$ として、
$$
 \sum_{n=0}^{N-1} \sum_{i=0}^{N-1} \sum_{j=0}^{N-1} d_{ij} q_{n,i} q_{n+1,j}
$$

In [ ]:
from amplify import sum_poly

cost = sum_poly(
    ncity,
    lambda n: sum_poly(
        ncity,
        lambda i: sum_poly(
            ncity, lambda j: distances[i, j] * q[n, i] * q[(n + 1) % ncity, j]
        ),
    ),
)

### 行の one-hot

In [ ]:
from amplify.constraint import one_hot

# 行に対する制約
row_constraints = [one_hot(q[n]) for n in range(ncity)]

### 列の one-hot

In [ ]:
# 列に対する制約
col_constraints = [one_hot(q[:, i]) for i in range(ncity)]

### 制約

In [ ]:
from amplify import sum_poly

constraints = sum(row_constraints) + sum(col_constraints)

constraints *= np.amax(distances)  # 制約条件の強さを設定

### イジングモデル生成

In [ ]:
#######################################################
# 確認用
model = cost + constraints
# model.logical_poly
# model.input_constraints
# model.input_constraints[0][0].penalty

### ソルバ生成

In [ ]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.token = "rSnIw4H95J8GuAKbvPxzG2PMOCCcFoZD"  #2022-11-27まで有効
client.parameters.timeout = 1000  # タイムアウト1秒

solver = Solver(client)

### ソルバ実行

In [ ]:
result = solver.solve(model)
if len(result) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

energy = result[0].energy
values = result[0].values
print('energy = ',energy)

q_values = q.decode(values)
# print('q_values = ',q_values)

print('Execution time =',solver.execution_time, '[msec]')

### 経路のプロット

In [ ]:
def show_route(route: list, distances: np.ndarray, locations: np.ndarray):

    ncity = len(route)
    path_length = sum(
        [distances[route[i]][route[(i + 1) % ncity]] for i in range(ncity)]
    )

    x = [i[0] for i in locations]
    y = [i[1] for i in locations]
    plt.figure(figsize=(7, 7))
    plt.title(f"path length: {path_length}")
    plt.xlabel("x")
    plt.ylabel("y")

    for i in range(ncity):
        r = route[i]
        n = route[(i + 1) % ncity]
        plt.plot([x[r], x[n]], [y[r], y[n]], "b-")
    plt.plot(x, y, "ro")
    plt.show()

    # return path_length

In [ ]:
route = np.where(np.array(q_values) == 1)[1]
show_route(route, distances, locations)